# Forest Fire Incidence in Northern Algeria: Proposal

In [ ]:
library(tidyverse)
library(tidymodels)
options(repr.matrix.max.rows = 8)

In [ ]:
set.seed(38)

## Introduction

Northern Algeria has high wildfire activity, experiencing 32 204 forest fires between 2001 and 2019 (Curt et. al., 2020). 

Daily temperature, relative humidity, rainfall, wind speed, and Fire Weather Index (FWI) data were collected in the Bejaia and Sidi Bel Abbes regions of Northern Algeria from June to September in 2012. FWI is divided into two categories (De Groot, 1987). Fuel moisture codes (FFMC, DMC, and DC) represent ratings of moisture for specific fuel depths. Fire behaviour indices include Initial Spread Index (ISI), expected rate of fire spread, Buildup Index (BUI), amount of fuel available, and Fire Weather Index, expected fire intensity.

Our goal is to answer: Can we predict whether a region in Northern Algeria will experience a forest fire based upon bioclimatic conditions?


## Preliminary Exploratory Data Analysis

In [ ]:
## Load datasets from URL

algerian_forest_fire_Bejaia <- read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00547/Algerian_forest_fires_dataset_UPDATE.csv", 
                                 skip = 1, n_max = 122) %>%
    mutate(Region = "Bejaia")

algerian_forest_fire_Sidi_Bel_Abbes <- read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00547/Algerian_forest_fires_dataset_UPDATE.csv",
                                               skip = 126) %>%
    mutate(Region = "Sidi Bel Abbes")

## Merge datasets of the two regions

algeria_forest_fire_all_data <- rbind(algerian_forest_fire_Bejaia, algerian_forest_fire_Sidi_Bel_Abbes) %>%
    mutate(Classes = as_factor(Classes)) %>%
    mutate(Region = as_factor(Region))

In [ ]:
## Manually set values of incorrect column, convert DC and FWI to numeric type

algeria_forest_fire_all_data[166, 10] <- '14.6'
algeria_forest_fire_all_data[166, 11] <- 9
algeria_forest_fire_all_data[166, 12] <- 12.5
algeria_forest_fire_all_data[166, 13] <- '10.4'
algeria_forest_fire_all_data[166, 14] <- "fire" 

algeria_forest_fire_all_data <- algeria_forest_fire_all_data %>%
    mutate(DC = as.numeric(DC)) %>%
    mutate(FWI = as.numeric(FWI))


In [ ]:
## Create training dataset

forest_fire_split <- algeria_forest_fire_all_data %>%
    select(Temperature, RH, Rain, FFMC, Classes) %>%
    initial_split(prop = 0.65, strata = Classes)
forest_fire_train <- training(forest_fire_split)
forest_fire_test <- testing(forest_fire_split)

forest_fire_train

The following table summarizes the predictors’ means and the number of observations for each of the classes. The classes are approximately balanced, with more observations of fire. Days with fire display higher mean temperatures, lower mean relative humidity, higher mean FFMC, and lower mean rainfall.

In [ ]:
## Explore means of predictor variables and the number of observations in each category

means <- forest_fire_train %>%
    group_by(Classes) %>%
    summarize(mean_temp = mean(Temperature),
             mean_RH = mean(RH),
             mean_rain = mean(Rain),
             mean_FFMC = mean(FFMC),
             obs_count = n())

means

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 6)

Next, we compare the distribution of the predictors between the 2 classes, examining differences which might suggest how each variable relates to fire incidence by constructing histograms for each predictor.

In [ ]:
# Plot distribution of 'temperature' predictor on days with and without fire
Classes.labs <- c("No Fire", "Fire")
names(Classes.labs) <- c("not fire", "fire")


temp_distribution_plot_fire <- forest_fire_train %>%
    ggplot(aes(x = Temperature)) +
    geom_histogram() +
    labs(x = "Temperature (degrees Celsius)", y = "Frequency", title = "Histogram of Temperature Measurements") +
    facet_grid(Classes ~ ., labeller = labeller(Classes = Classes.labs)) +
    theme(text = element_text(size = 20))

temp_distribution_plot_fire

This histogram supports that higher temperatures may be related to fire, since the distribution for the "fire" class is centered approximately 5°C higher.


In [ ]:
## Plot distribution of 'relative humidity' predictor on days with and without fire
Classes.labs <- c("No Fire", "Fire")
names(Classes.labs) <- c("not fire", "fire")

RH_distribution_plot_fire <- forest_fire_train %>%
    ggplot(aes(x = RH)) +
    geom_histogram() +
    labs(x = "Relative Humidity (%)", y = "Frequency", title = "Histogram of Relative Humidity Measurements") +
    facet_grid(Classes ~ ., labeller = labeller(Classes = Classes.labs)) +
    theme(text = element_text(size = 20))

RH_distribution_plot_fire

Days with no fire have consistently higher relative humidity, whereas relative humidity varies greatly on days with fires.

In [ ]:
## Plot distribution of 'rainfall' predictor on days with and without fire

rain_distribution_plot_fire <- forest_fire_train %>%
    ggplot(aes(x = Rain)) +
    geom_histogram() +
    labs(x = "Rainfall (mm)", y = "Frequency", title = "Histogram of Rainfall Measurements") +
    facet_grid(Classes ~ ., labeller = labeller(Classes = Classes.labs)) +
    theme(text = element_text(size = 20))

rain_distribution_plot_fire

Days with fire have minimal rainfall, whereas rainfall varies from 0 to 20 mm on days without fire.


In [ ]:
## Plot distribution of 'Fine Fuel Moisture Code' predictor on days with and without fire

FFMC_distribution_plot_fire <- forest_fire_train %>%
    ggplot(aes(x = FFMC)) +
    geom_histogram() +
    labs(x = "Fine Fuel Moisture Code (FFMC)", y = "Frequency", title = "Histogram of FFMC") +
    facet_grid(Classes ~ ., labeller = labeller(Classes = Classes.labs)) +
    theme(text = element_text(size = 20))

FFMC_distribution_plot_fire

For all classifications of “fire,” FFMC is greater than 80. This suggests that FFMC may be an excellent predictor of fire, with FFMC ≈ 80 separating the two classes.

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 6)

## Methods

We plan to use a k-nearest neighbours classification algorithm with cross-validation to predict whether there will be a fire on a given day. As BUI, ISI, and FWI, characterize fire behaviour, we decided that they would not be helpful in predicting the fire likelihood.  From the exploratory analysis, patterns appear in the values of temperature, humidity, rain, and FFMC based upon the fire classification; therefore, these will be the variables included in our algorithm. The results will be shown with scatterplots of the data, the predictors as axes, and colored by classification.

## Expected Outcomes and Significance

We anticipate that FFMC will be the strongest predictor of fire due to the distinct separation in values observed in the histogram. It is expected that higher temperature, lower humidity, and minimal rain correspond to an incident of fire. 

Our findings may assist in determining whether a fire will occur based on bioclimatic conditions. This knowledge could improve interventions to prevent forest fires, protecting the ecosystems and residents of these regions.

The causes of forest fires in Algeria are widely unknown, with the causation for 85% of forest fires remaining unknown in 2020 (Pons, 2020). However, our data analysis could lead to asking whether rainfall, temperature, humidity, or FFMC directly cause a fire incidence.

In [ ]:
set.seed(1)
fire_split <- initial_split(algeria_forest_fire_all_data, prop = 0.75, strata = Classes)
fire_train <- training(fire_split)
fire_test <- testing(fire_split)

fire_recipe <- recipe(Classes ~ FFMC + Temperature + Rain + RH, data = fire_train) %>%
    step_scale(all_predictors()) %>%
    step_center(all_predictors())

knn_tune_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
  set_engine("kknn") %>%
  set_mode("classification")

fire_vfold <- vfold_cv(fire_train, v = 15, strata = Classes)

tuning_results <- workflow() %>%
    add_recipe(fire_recipe) %>%
    add_model(knn_tune_spec) %>%
    tune_grid(resamples = fire_vfold, grid = 15) %>%
    collect_metrics()

accuracies <- tuning_results %>%
    filter(.metric == "accuracy")

accuracy_vs_k <- ggplot(accuracies, aes(x = neighbors, y = mean)) +
  geom_point() +
  geom_line() +
  labs(x = "Neighbors", y = "Accuracy Estimate")
accuracy_vs_k


In [ ]:
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 7) %>%
  set_engine("kknn") %>%
  set_mode("classification")

fire_fit <- workflow() %>%
    add_recipe(fire_recipe) %>%
    add_model(knn_spec) %>%
    fit(data = fire_train)

fire_fit

## References

Aini, A., Curt, T., Bekdouche, F. (2020). Modelling fire hazard in the southern Mediterranean fire rim (Bejaia region, northern Algeria). Retrieved March 03, 2021, from
https://link.springer.com/article/10.1007/s10661-019-7931-0#Sec1

Curt, T., Aini, A., Dupire, S. (2020). Fire activity in Mediterranean forests (The Algerian case). Retrieved March 03, 2021, from
https://www.mendeley.com/catalogue/0ad59940-c469-388c-8796-0685cacb334f/

De Groot, W. J. (1987, April 2). Interpreting the Canadian Forest Fire Weather Index (FWI) System. Retrieved March 03, 2021, from
http://www.dnr.state.mi.us/WWW/FMD/WEATHER/Reference/FWI_Background.pdf

Pons, J. (2020, June 29). Algeria declares land and space war on forest fires. Retrieved March 03, 2021, from https://atalayar.com/en/content/algeria-declares-land-and-space-war-forest-fires